## Bibliotecas

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

## Carregando os dados

In [2]:
dados = pd.read_csv('Dados/stackoverflow_perguntas.csv')
dados.head()

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular


## Analisando os dados

In [3]:
print(f'Linhas: {dados.shape[0]}')
print(f'Colunas: {dados.shape[1]}')

Linhas: 5408
Colunas: 2


Podemos verificar abaixo que na coluna Tags, temos 37 valores únicos.

In [4]:
print(len(dados['Tags'].unique()))

37


Porém, ao analisar cada um dos valores únicos, notamos existência de alguns valores que podemos tratar.

In [5]:
dados['Tags'].unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

## Tratamento de dados

### Criando uma lista com os tags separados
Abaixo, criamos uma lista de tags únicos existentes no nosso dataframe. Ao executar o código abaixo, podemos observar que há 4 tags diferentes: node.js, jquery, html e angular.

In [6]:
lista_tags = list()

for tags in dados['Tags'].unique():
    for tag in tags.split():
        if tag not in lista_tags:
            lista_tags.append(tag)

print(lista_tags)

['node.js', 'jquery', 'html', 'angular']


### Criando uma função para criar uma coluna para cada tag

Abaixo, criamos uma função para criar colunas de cada tag. Os dados são preeenchidos de acordo com a coluna Tags.

In [7]:
def nova_coluna(lista_tags, dataframe, nome_tags):
    for tag in lista_tags:
        coluna = list()
        for linha_tag in dataframe[nome_tags]:
            if tag in linha_tag:
                coluna.append(1)
            else:
                coluna.append(0)

        dataframe[tag] = coluna

nova_coluna(lista_tags, dados, 'Tags')

dados.head()

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1


### Zip para classificação multilabel

Por fim, juntamos os dados das 4 tags em uma coluna para facilitar a criação e o performance do modelo.

In [8]:
lista_zip = list(zip(dados[lista_tags[0]], 
                dados[lista_tags[1]],
                dados[lista_tags[2]],
                dados[lista_tags[3]],))

dados['todas_tags'] = lista_zip

dados.head()

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,"(0, 0, 1, 1)"


## Treinando o modelo

### Separando os dados de treino e teste

Separamos os dados da seguinte forma: a coluna Perguntas será a variável feature e a coluna todas_tags criado anteriormente será a variável target, 20% dos dados serão os dados de teste.

In [9]:
perguntas_treino, perguntas_teste, tags_treino, tags_teste = train_test_split(dados['Perguntas'], 
                                                                              dados['todas_tags'], 
                                                                              test_size = 0.2, 
                                                                              random_state = 123)

### Vetorização dos dados

Por estarmos trabalhando com variável de texto, vetorizamos a coluna.

In [10]:
# Instanciando o vetorizador.
vetorizador = TfidfVectorizer(max_features = 5000,
                              max_df = 0.85)

# Treinando o vetorizador.
vetorizador.fit(dados['Perguntas'])

# Vetorizando as perguntas de treino e teste.
perguntas_treino_vetorizadas = vetorizador.transform(perguntas_treino)
perguntas_teste_vetorizadas = vetorizador.transform(perguntas_teste)